In [ ]:
import pandas as pd
import os
desktop_path = os.path.join(os.path.expanduser("~"), "llm-justification-evaluation", "Data_cleaning_cosine_calculation_semantic_and_analysis")
os.chdir(desktop_path)

In [65]:
reading_comprehension_data = pd.read_csv('NLP_analysis/reading_comprehension_analysis.csv')
reading_comprehension_data

,QuestionID,deepseek-r1:1.5b,deepseek-r1:14b,qwen2.5:1.5b,qwen2.5:14b,deepseek-r1:1.5b_time,deepseek-r1:14b_time,qwen2.5:1.5b_time,qwen2.5:14b_time,Answer,...,qwen2.5:14b_reasoning,qwen2.5:14b_solution,qwen2.5:14b_structure_score,qwen2.5:1.5b_reasoning,qwen2.5:1.5b_solution,qwen2.5:1.5b_structure_score,deepseek-r1:1.5b_correct,deepseek-r1:14b_correct,qwen2.5:1.5b_correct,qwen2.5:14b_correct
0,RCQ1,## Reasoning\n\nThe question asks about a fact...,"## Reasoning\nThe question asks which factor, ...",## Reasoning\n\nThe question asks about Theory...,## Reasoning\nThe question asks about *support...,70.8,149.5,3.9,96.6,B,...,The question asks about *supporting ideas* fro...,B,0,"The question asks about Theory B, which states...",E,0,0,1,0,1
1,RCQ10,## Reasoning\nThe passage suggests that SCNs c...,## Reasoning\nThe question asks about the auth...,## Reasoning\n\nThe passage discusses research...,## Reasoning\nThe question asks about the role...,7.2,89.8,4.5,87.8,B,...,The question asks about the role of the suprac...,B,0,The passage discusses research that has shifte...,B,0,1,1,1,1
2,RCQ100,The primary purpose of the passage is to prese...,## Reasoning\nThe passage primarily addresses ...,## Reasoning\n\nThe passage discusses why seve...,## Reasoning\n\nThe primary purpose of the pas...,8.5,107.1,3.5,87.1,B,...,The primary purpose of the passage is to expla...,B,0,The passage discusses why seventeenth-century ...,B,0,1,1,1,1
3,RCQ101,The essay question challenges Gallagher's expl...,## Reasoning\nThe question asks us to identify...,## Reasoning Section:\nThe question asks about...,## Reasoning\nThe question asks us about under...,97.0,106.4,4.4,90.1,C,...,The question asks us about undermining Gallagh...,D,0,Section:\nThe question asks about the correct ...,D,0,0,0,0,0
4,RCQ102,The essay focuses on critiquing limiting infor...,The passage primarily discusses Frazier and Mo...,## Reasoning\n\nThe passage discusses improvem...,## Reasoning\nThe primary focus of this passag...,6.2,111.3,4.1,87.5,D,...,The primary focus of this passage is on descri...,D,0,The passage discusses improvements in medical ...,A,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,RCQ95,## Reasoning\nThe passage discusses Maurice Du...,## Reasoning\nThe question asks what the autho...,## Reasoning\n\n- The question asks about the ...,## Reasoning\nThe question asks us to infer ho...,84.2,108.9,4.4,97.7,B,...,The question asks us to infer how subsequent b...,B,0,- The question asks about the author's implica...,B,0,1,1,1,1
135,RCQ96,## Reasoning \nThe passage focuses on how Rob...,## Reasoning\nThe passage refers to Robert Fil...,## Reasoning\n\nThe passage discusses how seve...,## Reasoning\n\nThe passage discusses the rela...,7.5,108.0,5.3,97.5,D,...,The passage discusses the relationship between...,D,0,The passage discusses how seventeenth-century ...,A,0,0,0,0,1
136,RCQ97,## Reasoning\nThe passage argues that seventee...,## Reasoning\nThe question asks about the seve...,"## Reasoning\n\nThe question asks about the ""f...",## Reasoning\n\nThe question asks about the ma...,92.3,96.7,5.0,100.1,E,...,The question asks about the main idea regardin...,B,0,"The question asks about the ""forerunner"" statu...",C,0,0,1,0,0
137,RCQ98,The passage discusses Margaret Cavendish's dec...,## Reasoning\nThe question asks about Margaret...,## Reasoning\n\nThe passage discusses Royalist...,## Reasoning\n\nThe question asks about the mo...,6.3,103.0,5.2,88.0,E,...,The question asks about the motivation behind ...,E,0,The passage discusses Royalist ideology associ...,C,0,0,1,0,1


In [66]:
print(reading_comprehension_data.dtypes)

QuestionID                           object
deepseek-r1:1.5b                     object
deepseek-r1:14b                      object
qwen2.5:1.5b                         object
qwen2.5:14b                          object
deepseek-r1:1.5b_time               float64
deepseek-r1:14b_time                float64
qwen2.5:1.5b_time                   float64
qwen2.5:14b_time                    float64
Answer                               object
Solution                             object
PassageText                          object
QuestionText                         object
deepseek-r1:1.5b_reasoning           object
deepseek-r1:1.5b_solution            object
deepseek-r1:1.5b_structure_score      int64
deepseek-r1:14b_reasoning            object
deepseek-r1:14b_solution             object
deepseek-r1:14b_structure_score       int64
qwen2.5:14b_reasoning                object
qwen2.5:14b_solution                 object
qwen2.5:14b_structure_score           int64
qwen2.5:1.5b_reasoning          

In [67]:
columns_to_check = [
    'Solution',
    'deepseek-r1:1.5b_reasoning',
    'deepseek-r1:14b_reasoning',
    'qwen2.5:1.5b_reasoning',
    'qwen2.5:14b_reasoning'
]

for col in columns_to_check:
    reading_comprehension_data[col] = reading_comprehension_data[col].astype(str)


In [68]:
corpus = reading_comprehension_data["Solution"].tolist() + \
         reading_comprehension_data["deepseek-r1:1.5b_reasoning"].tolist() + \
         reading_comprehension_data["deepseek-r1:14b_reasoning"].tolist() + \
         reading_comprehension_data["qwen2.5:1.5b_reasoning"].tolist() + \
         reading_comprehension_data["qwen2.5:14b_reasoning"].tolist()

In [69]:
corpus = [str(x).strip() if pd.notna(x) else "" for x in corpus]

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
## Analyze only words, min_df <- words appears at least in 1% of the documents,
## max_df <- words appears in at most 75% of the documents, sublinear_tf <- use sublinear term frequency scaling 1 + log(tf)
vectorizer = TfidfVectorizer(analyzer='word', min_df=0.01, max_df=0.75, ngram_range=(1, 2), stop_words='english', sublinear_tf=True)
X = vectorizer.fit_transform(corpus)

In [71]:
n = len(reading_comprehension_data)
Solution = X[0:n]
deepseek_r1_1_5b = X[n:2*n]
deepseek_r1_14b = X[2*n:3*n]
qwen2_5_1_5b = X[3*n:4*n]
qwen2_5_14b = X[4*n:5*n]

In [72]:
from sklearn.metrics.pairwise import cosine_similarity

deepseek_r1_1_5b_similarity = cosine_similarity(Solution, deepseek_r1_1_5b)
deepseek_r1_14b_similarity = cosine_similarity(Solution, deepseek_r1_14b)   
qwen2_5_1_5b_similarity = cosine_similarity(Solution, qwen2_5_1_5b)
qwen2_5_14b_similarity = cosine_similarity(Solution, qwen2_5_14b)

In [73]:
## The result is a matrix where the cell (i,j) contains the cosine similarity between the i-th solution and the j-th reasoning.
## But I want only the cosine similarity between the i-th solution and the i-th reasoning.
deepseek_r1_1_5b_similarity = deepseek_r1_1_5b_similarity.diagonal()
deepseek_r1_14b_similarity = deepseek_r1_14b_similarity.diagonal()
qwen2_5_1_5b_similarity = qwen2_5_1_5b_similarity.diagonal()
qwen2_5_14b_similarity = qwen2_5_14b_similarity.diagonal()

In [74]:
mean_deepseek_r1_1_5b_similarity = deepseek_r1_1_5b_similarity.mean()
mean_deepseek_r1_14b_similarity = deepseek_r1_14b_similarity.mean()
mean_qwen2_5_1_5b_similarity = qwen2_5_1_5b_similarity.mean()
mean_qwen2_5_14b_similarity = qwen2_5_14b_similarity.mean()

In [75]:
print("DeepSeek R1 1.5B Similarity:")
print(deepseek_r1_1_5b_similarity.mean(axis=0))
print("DeepSeek R1 14B Similarity:")
print(deepseek_r1_14b_similarity.mean(axis=0))
print("Qwen2.5 1.5B Similarity:")
print(qwen2_5_1_5b_similarity.mean(axis=0))
print("Qwen2.5 14B Similarity:")
print(qwen2_5_14b_similarity.mean(axis=0))

DeepSeek R1 1.5B Similarity:
0.3288909134730664
DeepSeek R1 14B Similarity:
0.4488498063712415
Qwen2.5 1.5B Similarity:
0.43041686310613525
Qwen2.5 14B Similarity:
0.4823469394828843


In [76]:
import numpy as np


stats = {
    "Model": [
        "DeepSeek R1 1.5B",
        "DeepSeek R1 14B",
        "Qwen2.5 1.5B",
        "Qwen2.5 14B"
    ],
    "Mean": [
        np.mean(deepseek_r1_1_5b_similarity),
        np.mean(deepseek_r1_14b_similarity),
        np.mean(qwen2_5_1_5b_similarity),
        np.mean(qwen2_5_14b_similarity)
    ],
    "Median": [
        np.median(deepseek_r1_1_5b_similarity),
        np.median(deepseek_r1_14b_similarity),
        np.median(qwen2_5_1_5b_similarity),
        np.median(qwen2_5_14b_similarity)
    ],
    "Std": [
        np.std(deepseek_r1_1_5b_similarity),
        np.std(deepseek_r1_14b_similarity),
        np.std(qwen2_5_1_5b_similarity),
        np.std(qwen2_5_14b_similarity)
    ]
}

stats_df = pd.DataFrame(stats)
print(stats_df)

              Model      Mean    Median       Std
0  DeepSeek R1 1.5B  0.328891  0.327270  0.139714
1   DeepSeek R1 14B  0.448850  0.454555  0.121297
2      Qwen2.5 1.5B  0.430417  0.425027  0.117002
3       Qwen2.5 14B  0.482347  0.488476  0.101821


In [77]:
deepseek_r1_1_5b_similarity_crit = pd.read_csv('vector_exports/deepseek_r1_1_5b_similarity_crit.csv', header=None)
deepseek_r1_14b_similarity_crit = pd.read_csv('vector_exports/deepseek_r1_14b_similarity_crit.csv', header=None)
qwen2_5_1_5b_similarity_crit = pd.read_csv('vector_exports/qwen2_5_1_5b_similarity_crit.csv', header=None)
qwen2_5_14b_similarity_crit = pd.read_csv('vector_exports/qwen2_5_14b_similarity_crit.csv', header=None)

In [78]:
import numpy as np

deepseek_r1_1_5b_similarity_crit_values = deepseek_r1_1_5b_similarity_crit.iloc[:, 0].values
deepseek_r1_14b_similarity_crit_values = deepseek_r1_14b_similarity_crit.iloc[:, 0].values
qwen2_5_1_5b_similarity_crit_values = qwen2_5_1_5b_similarity_crit.iloc[:, 0].values
qwen2_5_14b_similarity_crit_values = qwen2_5_14b_similarity_crit.iloc[:, 0].values

deepseek_r1_1_5b_similarity_all = np.concatenate([deepseek_r1_1_5b_similarity, deepseek_r1_1_5b_similarity_crit_values])
deepseek_r1_14b_similarity_all = np.concatenate([deepseek_r1_14b_similarity, deepseek_r1_14b_similarity_crit_values])
qwen2_5_1_5b_similarity_all = np.concatenate([qwen2_5_1_5b_similarity, qwen2_5_1_5b_similarity_crit_values])
qwen2_5_14b_similarity_all = np.concatenate([qwen2_5_14b_similarity, qwen2_5_14b_similarity_crit_values])

In [79]:
import numpy as np


stats = {
    "Model": [
        "DeepSeek R1 1.5B",
        "DeepSeek R1 14B",
        "Qwen2.5 1.5B",
        "Qwen2.5 14B"
    ],
    "Mean": [
        np.mean(deepseek_r1_1_5b_similarity_all),
        np.mean(deepseek_r1_14b_similarity_all),
        np.mean(qwen2_5_1_5b_similarity_all),
        np.mean(qwen2_5_14b_similarity_all)
    ],
    "Median": [
        np.median(deepseek_r1_1_5b_similarity_all),
        np.median(deepseek_r1_14b_similarity_all),
        np.median(qwen2_5_1_5b_similarity_all),
        np.median(qwen2_5_14b_similarity_all)
    ],
    "Std": [
        np.std(deepseek_r1_1_5b_similarity_all),
        np.std(deepseek_r1_14b_similarity_all),
        np.std(qwen2_5_1_5b_similarity_all),
        np.std(qwen2_5_14b_similarity_all)
    ]
}

stats_df = pd.DataFrame(stats)
print(stats_df)

              Model      Mean    Median       Std
0  DeepSeek R1 1.5B  0.330618  0.332665  0.120570
1   DeepSeek R1 14B  0.406491  0.407755  0.112154
2      Qwen2.5 1.5B  0.425395  0.427978  0.110051
3       Qwen2.5 14B  0.461861  0.466680  0.100458


N-GRAMS OVERLAP

In [80]:
def ngrams(text, n):
    tokens = [str(t).lower() for t in str(text).split()]
    return set([' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)])

def ngram_overlap(text1, text2, n):
    ngrams1 = ngrams(text1, n)
    ngrams2 = ngrams(text2, n)
    
    intersection = len(ngrams1 & ngrams2)
    union = len(ngrams1 | ngrams2)
    
    if union == 0:
        return 0.0
    else:
        return intersection / union


In [81]:
# N-gram overlap with n=3 (can change with n=2, n=1)
n = 3

reading_comprehension_data['deepseek_r1_1_5b_ngram_overlap'] = reading_comprehension_data.apply(lambda row: ngram_overlap(row['Solution'], row['deepseek-r1:1.5b_reasoning'], n), axis=1)
reading_comprehension_data['deepseek_r1_14b_ngram_overlap']  = reading_comprehension_data.apply(lambda row: ngram_overlap(row['Solution'], row['deepseek-r1:14b_reasoning'], n), axis=1)
reading_comprehension_data['qwen2_5_1_5b_ngram_overlap']     = reading_comprehension_data.apply(lambda row: ngram_overlap(row['Solution'], row['qwen2.5:1.5b_reasoning'], n), axis=1)
reading_comprehension_data['qwen2_5_14b_ngram_overlap']      = reading_comprehension_data.apply(lambda row: ngram_overlap(row['Solution'], row['qwen2.5:14b_reasoning'], n), axis=1)

In [82]:
mean_deepseek_r1_1_5b_ngram = reading_comprehension_data['deepseek_r1_1_5b_ngram_overlap'].mean()
mean_deepseek_r1_14b_ngram  = reading_comprehension_data['deepseek_r1_14b_ngram_overlap'].mean()
mean_qwen2_5_1_5b_ngram     = reading_comprehension_data['qwen2_5_1_5b_ngram_overlap'].mean()
mean_qwen2_5_14b_ngram      = reading_comprehension_data['qwen2_5_14b_ngram_overlap'].mean()

In [83]:
print("Mean N-gram overlap (DeepSeek 1.5B):", mean_deepseek_r1_1_5b_ngram)
print("Mean N-gram overlap (DeepSeek 14B):", mean_deepseek_r1_14b_ngram)
print("Mean N-gram overlap (Qwen 1.5B):", mean_qwen2_5_1_5b_ngram)
print("Mean N-gram overlap (Qwen 14B):", mean_qwen2_5_14b_ngram)

Mean N-gram overlap (DeepSeek 1.5B): 0.007851814026159669
Mean N-gram overlap (DeepSeek 14B): 0.01973989881883749
Mean N-gram overlap (Qwen 1.5B): 0.016581251422474692
Mean N-gram overlap (Qwen 14B): 0.02803851749545408


METRICHE / SCORE

In [84]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import single_meteor_score
from rouge_score import rouge_scorer
import sacrebleu

In [85]:
### BLEU SCORE ###

#smoothing function in order to prevent zero division errors
smoothie = SmoothingFunction().method4

def compute_bleu(reference_text, candidate_text):
    reference = [reference_text.lower().split()]
    candidate = candidate_text.lower().split()
    return sentence_bleu(reference, candidate, smoothing_function=smoothie)

In [86]:
### ROUGE-L ###

rouge_scorer_obj = rouge_scorer.RougeScorer(['rougeL'], use_stemmer = True)

def compute_rouge(reference_text, candidate_text):
    scores = rouge_scorer_obj.score(reference_text, candidate_text)
    #F1 measure of ROUGE-L
    return scores['rougeL'].fmeasure 

In [87]:
### METEOR SCORE ###

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

def compute_meteor(reference_text, candidate_text):
    return single_meteor_score(reference_text.lower().split(), candidate_text.lower().split())

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [88]:
# BLEU
reading_comprehension_data['deepseek_r1_1_5b_bleu'] = reading_comprehension_data.apply(lambda row: compute_bleu(row['Solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
reading_comprehension_data['deepseek_r1_14b_bleu']  = reading_comprehension_data.apply(lambda row: compute_bleu(row['Solution'], row['deepseek-r1:14b_reasoning']), axis=1)
reading_comprehension_data['qwen2_5_1_5b_bleu']     = reading_comprehension_data.apply(lambda row: compute_bleu(row['Solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
reading_comprehension_data['qwen2_5_14b_bleu']      = reading_comprehension_data.apply(lambda row: compute_bleu(row['Solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [89]:
# ROUGE-L
reading_comprehension_data['deepseek_r1_1_5b_rouge_l'] = reading_comprehension_data.apply(lambda row: compute_rouge(row['Solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
reading_comprehension_data['deepseek_r1_14b_rouge_l']  = reading_comprehension_data.apply(lambda row: compute_rouge(row['Solution'], row['deepseek-r1:14b_reasoning']), axis=1)
reading_comprehension_data['qwen2_5_1_5b_rouge_l']     = reading_comprehension_data.apply(lambda row: compute_rouge(row['Solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
reading_comprehension_data['qwen2_5_14b_rouge_l']      = reading_comprehension_data.apply(lambda row: compute_rouge(row['Solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [90]:
# METEOR
reading_comprehension_data['deepseek_r1_1_5b_meteor'] = reading_comprehension_data.apply(lambda row: compute_meteor(row['Solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
reading_comprehension_data['deepseek_r1_14b_meteor']  = reading_comprehension_data.apply(lambda row: compute_meteor(row['Solution'], row['deepseek-r1:14b_reasoning']), axis=1)
reading_comprehension_data['qwen2_5_1_5b_meteor']     = reading_comprehension_data.apply(lambda row: compute_meteor(row['Solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
reading_comprehension_data['qwen2_5_14b_meteor']      = reading_comprehension_data.apply(lambda row: compute_meteor(row['Solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [91]:
print("Mean BLEU DeepSeek_1.5:", reading_comprehension_data['deepseek_r1_1_5b_bleu'].mean())
print("Mean BLEU DeepSeek_14B:", reading_comprehension_data['deepseek_r1_14b_bleu'].mean())
print("Mean BLEU Qwen2.5_1.5:", reading_comprehension_data['qwen2_5_1_5b_bleu'].mean())
print("Mean BLEU Qwen2.5_14B:", reading_comprehension_data['qwen2_5_14b_bleu'].mean())

Mean BLEU DeepSeek_1.5: 0.0048526136028382415
Mean BLEU DeepSeek_14B: 0.04135390590936408
Mean BLEU Qwen2.5_1.5: 0.0492739300547414
Mean BLEU Qwen2.5_14B: 0.0815409723109048


In [92]:
print("Mean ROUGE-L DeepSeek_1.5:", reading_comprehension_data['deepseek_r1_1_5b_rouge_l'].mean())
print("Mean ROUGE-L DeepSeek_14B:", reading_comprehension_data['deepseek_r1_14b_rouge_l'].mean())
print("Mean ROUGE-L Qwen2.5_1.5:", reading_comprehension_data['qwen2_5_1_5b_rouge_l'].mean())
print("Mean ROUGE-L Qwen2.5_14B:", reading_comprehension_data['qwen2_5_14b_rouge_l'].mean())

Mean ROUGE-L DeepSeek_1.5: 0.1407009571399458
Mean ROUGE-L DeepSeek_14B: 0.2267232911618398
Mean ROUGE-L Qwen2.5_1.5: 0.22828346162287502
Mean ROUGE-L Qwen2.5_14B: 0.26138821780919386


In [93]:
print("Mean METEOR DeepSeek_1.5:", reading_comprehension_data['deepseek_r1_1_5b_meteor'].mean())
print("Mean METEOR DeepSeek_14B:", reading_comprehension_data['deepseek_r1_14b_meteor'].mean())
print("Mean METEOR Qwen2.5_1.5:", reading_comprehension_data['qwen2_5_1_5b_meteor'].mean())
print("Mean METEOR Qwen2.5_14B:", reading_comprehension_data['qwen2_5_14b_meteor'].mean())

Mean METEOR DeepSeek_1.5: 0.08365382633599032
Mean METEOR DeepSeek_14B: 0.18493271359358818
Mean METEOR Qwen2.5_1.5: 0.23500635224193359
Mean METEOR Qwen2.5_14B: 0.3129389299611956


CONCLUSION

In [94]:
## Create a DataFrame to summarize the metrics

metrics = {
    "Cosine Similarity": [
        deepseek_r1_1_5b_similarity.mean(axis=0),
        deepseek_r1_14b_similarity.mean(axis=0),
        qwen2_5_1_5b_similarity.mean(axis=0),
        qwen2_5_14b_similarity.mean(axis=0)
    ],
    "N-gram Overlap": [
        reading_comprehension_data['deepseek_r1_1_5b_ngram_overlap'].mean(),
        reading_comprehension_data['deepseek_r1_14b_ngram_overlap'].mean(),
        reading_comprehension_data['qwen2_5_1_5b_ngram_overlap'].mean(),
        reading_comprehension_data['qwen2_5_14b_ngram_overlap'].mean()
    ],
    "BLEU": [
        reading_comprehension_data['deepseek_r1_1_5b_bleu'].mean(),
        reading_comprehension_data['deepseek_r1_14b_bleu'].mean(),
        reading_comprehension_data['qwen2_5_1_5b_bleu'].mean(),
        reading_comprehension_data['qwen2_5_14b_bleu'].mean()
    ],
    "ROUGE-L": [
        reading_comprehension_data['deepseek_r1_1_5b_rouge_l'].mean(),
        reading_comprehension_data['deepseek_r1_14b_rouge_l'].mean(),
        reading_comprehension_data['qwen2_5_1_5b_rouge_l'].mean(),
        reading_comprehension_data['qwen2_5_14b_rouge_l'].mean()
    ],
    "METEOR": [
        reading_comprehension_data['deepseek_r1_1_5b_meteor'].mean(),
        reading_comprehension_data['deepseek_r1_14b_meteor'].mean(),
        reading_comprehension_data['qwen2_5_1_5b_meteor'].mean(),
        reading_comprehension_data['qwen2_5_14b_meteor'].mean()
    ]
}

models = ["DeepSeek R1 1.5B", "DeepSeek R1 14B", "Qwen2.5 1.5B", "Qwen2.5 14B"]
metrics_df = pd.DataFrame(metrics, index=models)
print(metrics_df)

                  Cosine Similarity  N-gram Overlap      BLEU   ROUGE-L  \
DeepSeek R1 1.5B           0.328891        0.007852  0.004853  0.140701   
DeepSeek R1 14B            0.448850        0.019740  0.041354  0.226723   
Qwen2.5 1.5B               0.430417        0.016581  0.049274  0.228283   
Qwen2.5 14B                0.482347        0.028039  0.081541  0.261388   

                    METEOR  
DeepSeek R1 1.5B  0.083654  
DeepSeek R1 14B   0.184933  
Qwen2.5 1.5B      0.235006  
Qwen2.5 14B       0.312939  
